13307130167 刘阳

In [1]:
import nltk

### Assignment 1: 名字性别分类器

In [3]:
from nltk.corpus import names
labeled_names = [(name, 'male') for name in names.words('male.txt')] + \
                [(name, 'female') for name in names.words('female.txt')]
import random
random.shuffle(labeled_names)

In [4]:
train_names = labeled_names[800:]
devtest_names = labeled_names[400:800]
test_names = labeled_names[:400]

In [5]:
def gender_features(name):
    return {'suffix1': name[-1:], 'suffix2': name[-2:]}

In [21]:
def evaluate_feature_function(feature_func, train_names, devtest_names):
    train_set = [(feature_func(n), g) for (n, g) in train_names]
    devtest_set = [(feature_func(n), g) for (n, g) in devtest_names]
    classifier = nltk.NaiveBayesClassifier.train(train_set)
    accuracy = nltk.classify.accuracy(classifier, devtest_set)
    return accuracy

In [22]:
print evaluate_feature_function(gender_features, train_names, devtest_names)

0.78


#### add more \*fix features

In [23]:
def gender_features_v1(name):
    return {
        'prefix1': name[:1],
        'prefix2': name[:2],
        'suffix1': name[-1:],
        'suffix2': name[-2:]
    }

In [24]:
print evaluate_feature_function(gender_features_v1, train_names, devtest_names)

0.7925


#### add '\*son' feature

In [35]:
def gender_features_v2(name):
    result = gender_features_v1(name)
    result['contains"son"'] = name.lower().find('son') != -1
    return result

In [36]:
print evaluate_feature_function(gender_features_v2, train_names, devtest_names)

0.7925


#### add bag of chars feature

In [37]:
def gender_features_v3(name):
    return dict([(c, True) for c in name])

In [38]:
print evaluate_feature_function(gender_features_v3, train_names, devtest_names)

0.7225


#### add bag of nchars feature

In [39]:
def gender_features_v4(name, n=2):
    ncs = [name[i:i+2] for i in range(len(name)-1)]
    return gender_features_v3(ncs)

In [40]:
print evaluate_feature_function(gender_features_v4, train_names, devtest_names)

0.725


#### mix above ways

In [41]:
def gender_features_v5(name):
    result = dict()
    result.update(gender_features_v2(name))
    result.update(gender_features_v3(name))
    result.update(gender_features_v4(name))
    return result

In [42]:
print evaluate_feature_function(gender_features_v5, train_names, devtest_names)

0.805


#### test verson 5 function on test data set

In [43]:
train_set = [(gender_features_v5(n), g) for (n, g) in train_names]
test_set = [(gender_features_v5(n), g) for (n, g) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
accuracy = nltk.classify.accuracy(classifier, test_set)
print accuracy

0.81


在测试集上的性能（0.81）比在开发集上的性能（0.805）要好

### Assignment: 特定特征信息量解读

最有信息量的特征：
- '==True and label is 'neg'
- also==True and label is 'neg'
- looks==True and label is 'neg'
- ('by', 'a')==True and label is 'neg'
- waste==True and label is 'neg'
- for==True and label is 'neg'
- makes==True and label is 'neg'
- have==True and label is 'neg'
- ('plot', ',')==True and label is 'neg'
- ,==True and label is 'neg'

从上述特征量列表中可以看出：
- 'looks' 这样的词一般表示轻微转折的语气，所以更多出现在负面评价中
- 'waste' 显然是在讲'waste time', 'waste money',这样的评价是很负面的评价
- 'plot' 讲故事情节，也会和电影评价有关